# MCDA Neonatal Mouse Heart

In [ ]:
from typing import Tuple
from matplotlib.animation import FuncAnimation

import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
import biorsp as rsp

In [2]:
rsp.init(
    {
        "MODE": "absolute",
        "SCANNING_WINDOW": np.pi,
        "RESOLUTION": 100,
    }
)

In [ ]:
dge_matrix = rsp.preprocess_dge_matrix("data/Neonatalheart_1_dge.txt")

In [ ]:
tsne_embedding = rsp.compute_tsne(dge_matrix)

plt.scatter(tsne_embedding[:, 0], tsne_embedding[:, 1], s=1)
plt.show()

In [ ]:
dbscan_labels = rsp.compute_dbscan(tsne_embedding)

for label in np.unique(dbscan_labels):
    plt.scatter(
        tsne_embedding[dbscan_labels == label, 0],
        tsne_embedding[dbscan_labels == label, 1],
        s=1,
        label="Cluster {}".format(label) if label != 0 else "Noise",
    )

plt.legend(fontsize="small", loc="upper right")
plt.show()

In [6]:
def main(
    SCANNING_WINDOW: float,
    RESOLUTION: int,
    background_points: np.ndarray,
    foreground_points: np.ndarray,
    gif_filename: str = "radar_scanning_animation.gif",
) -> None:
    """
    Main function that takes parameters and point sets as input, converts them to polar coordinates,
    and animates three panels:
    1. The scanning window (polar plot).
    2. The in-progress RSP Diff in absolute mode (polar plot).
    3. The in-progress RSP Diff in relative mode (polar plot).

    Parameters
    ----------
    SCANNING_WINDOW : float
        Angular width of the scanning window in radians.
    RESOLUTION : int
        Number of frames (angles) in the 360-degree rotation.
    background_points : np.ndarray
        An array of shape (N, 2) for N background points (x, y).
    foreground_points : np.ndarray
        An array of shape (M, 2) for M foreground points (x, y).
    gif_filename : str
        Filename for the output GIF animation.
    """

    vantage_point: Tuple[float, float] = tuple(np.mean(background_points, axis=0))
    r_fg, theta_fg = rsp.cartesian_to_polar(foreground_points, vantage_point)
    r_bg, theta_bg = rsp.cartesian_to_polar(background_points, vantage_point)

    BIN_SIZE = SCANNING_WINDOW / RESOLUTION
    BIN_EDGES = np.arange(
        -SCANNING_WINDOW / 2, SCANNING_WINDOW / 2 + BIN_SIZE, BIN_SIZE
    )
    BIN_EDGES_DEG = np.degrees(BIN_EDGES)

    # Generate angles from 0 to 2π, inclusive, so final frame is full circle
    theta_k_list = np.linspace(0, 2 * np.pi, RESOLUTION, endpoint=True)

    radius_max: float = max(np.max(r_fg), np.max(r_bg))
    rsp_diffs_abs = np.zeros(RESOLUTION)
    rsp_diffs_rel = np.zeros(RESOLUTION)

    fig: plt.Figure = plt.figure(figsize=(18, 6))
    # Three panels, all polar
    ax1: plt.Axes = plt.subplot(1, 3, 1, projection="polar")
    ax2: plt.Axes = plt.subplot(1, 3, 2, projection="polar")
    ax3: plt.Axes = plt.subplot(1, 3, 3, projection="polar")

    def plot_scanning_window(
        ax: plt.Axes, start_angle: float, end_angle: float, radius_max: float
    ) -> None:
        if start_angle < end_angle:
            theta_window: np.ndarray = np.linspace(start_angle, end_angle, 100)
        else:
            theta_window: np.ndarray = np.linspace(
                start_angle, end_angle + 2 * np.pi, 100
            ) % (2 * np.pi)
        r_window: np.ndarray = np.full_like(theta_window, radius_max)
        theta_polygon: np.ndarray = np.concatenate(
            ([start_angle], theta_window, [end_angle], [start_angle])
        )
        r_polygon: np.ndarray = np.concatenate(([0], r_window, [0], [0]))
        ax.fill(
            theta_polygon, r_polygon, color="yellow", alpha=0.1, label="Scanning Window"
        )

    def animate(i: int) -> None:
        angle: float = theta_k_list[i]
        start_angle: float = (angle - SCANNING_WINDOW / 2) % (2 * np.pi)
        end_angle: float = (angle + SCANNING_WINDOW / 2) % (2 * np.pi)

        ax1.clear()
        ax1.set_title("Scanning Window", va="bottom")
        ax1.set_ylim(0, radius_max * 1.1)
        ax1.grid(True)

        ax1.scatter(theta_bg, r_bg, color="grey", s=1, label="Background", alpha=0.25)
        ax1.scatter(theta_fg, r_fg, color="red", s=1, label="Foreground", alpha=0.5)

        plot_scanning_window(ax1, start_angle, end_angle, radius_max)

        fg_in_window = rsp.within_angle(theta_fg, angle, SCANNING_WINDOW)
        bg_in_window = rsp.within_angle(theta_bg, angle, SCANNING_WINDOW)
        num_fg_in: int = np.sum(fg_in_window)
        num_bg_in: int = np.sum(bg_in_window)

        ax1.scatter(
            theta_bg[bg_in_window],
            r_bg[bg_in_window],
            color="grey",
            s=1,
            alpha=0.75,
            label="BG in Window",
        )
        ax1.scatter(
            theta_fg[fg_in_window],
            r_fg[fg_in_window],
            color="red",
            s=1,
            alpha=1.0,
            label="FG in Window",
        )

        handles, labels = ax1.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        ax1.legend(
            by_label.values(),
            by_label.keys(),
            loc="upper right",
            bbox_to_anchor=(1.1, 1.1),
        )

        fg_angles_in_window = theta_fg[fg_in_window]
        bg_angles_in_window = theta_bg[bg_in_window]

        # Compute relative angles
        relative_theta_fg = (fg_angles_in_window - angle + np.pi) % (2 * np.pi) - np.pi
        relative_theta_bg = (bg_angles_in_window - angle + np.pi) % (2 * np.pi) - np.pi

        fg_angles_shifted_in_window = relative_theta_fg[
            (relative_theta_fg >= -SCANNING_WINDOW / 2)
            & (relative_theta_fg <= SCANNING_WINDOW / 2)
        ]
        bg_angles_shifted_in_window = relative_theta_bg[
            (relative_theta_bg >= -SCANNING_WINDOW / 2)
            & (relative_theta_bg <= SCANNING_WINDOW / 2)
        ]

        fg_angles_shifted_deg = np.degrees(fg_angles_shifted_in_window)
        bg_angles_shifted_deg = np.degrees(bg_angles_shifted_in_window)

        # Compute histogram and CDFs (same for both absolute and relative)
        fg_hist, bg_hist = rsp.compute_histogram(
            fg_angles_shifted_deg, bg_angles_shifted_deg, BIN_EDGES_DEG
        )
        fg_cdf, bg_cdf = rsp.compute_cdfs(fg_hist, bg_hist)

        # Compute absolute mode diff
        fg_cdf_abs = fg_cdf.copy()
        if num_bg_in > 0:
            coverage_in_window = num_fg_in / num_bg_in
            # For absolute mode, scale FG CDF
            fg_cdf_abs *= coverage_in_window
        diff_abs = rsp.compute_diff(fg_cdf_abs, bg_cdf, mode="absolute")
        rsp_diffs_abs[i] = diff_abs

        # Compute relative mode diff
        # For relative mode, do not scale FG CDF
        diff_rel = rsp.compute_diff(fg_cdf, bg_cdf, mode="relative")
        rsp_diffs_rel[i] = diff_rel

        ax2.clear()
        ax2.set_title("RSP-a Plot")
        angles = theta_k_list[: i + 1]
        ax2.plot(angles, rsp_diffs_abs[: i + 1], color="black")
        ax2.grid(True)
        ax2.set_rlabel_position(0)
        ax2.set_ylim(0, 1)

        ax3.clear()
        ax3.set_title("RSP-r Plot")
        ax3.plot(angles, rsp_diffs_rel[: i + 1], label="RSP Diff (Rel)", color="black")
        ax3.legend(loc="upper right")
        ax3.grid(True)
        ax3.set_rlabel_position(0)
        ax3.set_ylim(0, 1)

    ani: FuncAnimation = FuncAnimation(
        fig, animate, frames=RESOLUTION, interval=100, repeat=False
    )
    ani.save(gif_filename, writer="pillow")

    plt.tight_layout()
    plt.show()

In [ ]:
SCANNING_WINDOW = rsp.get_param("SCANNING_WINDOW")
RESOLUTION = rsp.get_param("RESOLUTION")
MODE = rsp.get_param("MODE")

results = []

# Example genes to iterate over
for gene in ["Atf3", "Jun"]:
    gene_expression = dge_matrix.loc[gene].values

    # Foreground: cells where gene_expression > 0
    fg_indices = np.where(gene_expression > 0)[0]
    fg_points = tsne_embedding[fg_indices]

    # Background: all points in t-SNE embedding
    bg_points = tsne_embedding

    print(f"Gene {gene}: {len(fg_points)} foreground points, {len(bg_points)} background points")

    if len(fg_points) == 0:
        print(f"No foreground points for gene {gene}, skipping animation.")
        continue

    main(
        SCANNING_WINDOW=SCANNING_WINDOW,
        RESOLUTION=RESOLUTION,
        background_points=bg_points,
        foreground_points=fg_points,
        gif_filename=f"{gene}.gif"
    )